In [73]:
import os
import ffmpeg
from datetime import timedelta
import yaml
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import product

In [2]:
def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config('config.yaml')

In [3]:
dataframe = config['data_paths']['output_file_full_dataframe']
df = pd.read_csv(dataframe)

In [4]:
unique_secondary_types_separated = pd.Series([
    word.strip()
    for sublist in df['secondary_types'].dropna().str.split(',')
    for word in sublist
]).unique()
unique_secondary_types_separated

array(['back_pass', 'short_or_medium_pass', 'lateral_pass',
       'forward_pass', 'progressive_pass', 'loose_ball_duel', 'carry',
       'long_pass', 'pass_to_final_third', 'opportunity', 'head_pass',
       'recovery', 'under_pressure', 'ball_out', 'loss', 'hand_pass',
       'foul_suffered', 'ground_duel', 'linkup_play', 'offensive_duel',
       'defensive_duel', 'foul', 'dribble', 'aerial_duel',
       'counterpressing_recovery', 'pass', 'deep_completion',
       'touch_in_box', 'interception', 'sliding_tackle',
       'progressive_run', 'smart_pass', 'cross', 'deep_completed_cross',
       'pass_to_penalty_area', 'second_assist', 'assist', 'key_pass',
       'shot_assist', 'goal', 'conceded_goal', 'dribbled_past_attempt',
       'shot_after_throw_in', 'shot_block', 'free_kick_cross', 'save',
       'save_with_reflex', 'shot_after_corner', 'yellow_card',
       'cross_blocked', 'through_pass', 'free_kick_shot', 'shot',
       'head_shot', 'whistle', 'shot_after_free_kick', 'acceler

In [12]:
for secondary_type in unique_secondary_types_separated:
    df[secondary_type] = 0

In [13]:
# Populate the columns based on 'secondary_types' values
for index, row in df.iterrows():
    if pd.notna(row['secondary_types']):  # Check for NaN
        types = row['secondary_types'].split(',')
        types = [word.replace(' ', '') for word in types]
        for t in types:
            if t in unique_secondary_types_separated:
                df.at[index, t] = 1

In [15]:
pd.options.display.max_columns = 200

In [20]:
unique_types = df['primary_type'].unique()
dfs = {}
for typ in unique_types:
    dfs[typ] = df[df['primary_type'] == typ]

,Unnamed: 0,event_id,match_id,match_period,minute,second,match_timestamp,video_timestamp,primary_type,secondary_types,location_x,location_y,team_id,team_name,team_formation,opponent_team_id,opponent_team_name,opponent_team_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_length,recipient_id,recipient_name,recipient_position,end_location_x,end_location_y,shot_body_part,is_goal,on_target,goal_zone,xg,post_shot_xg,goalkeeper_action_id,goalkeeper,opponent_id_ground_duel,opponent_name_ground_duel,opponent_position_ground_duel,duel_type_ground_duel,kept_possession_ground_duel,progressed_with_ball_ground_duel,stopped_progress_ground_duel,recovered_possession_ground_duel,take_on_ground_duel,side_ground_duel,related_duel_id_ground_duel,opponent_id_aerial_duel,opponent_name_aerial_duel,opponent_position_aerial_duel,opponent_height_aerial_duel,first_touch_aerial_duel,height_aerial_duel,related_duel_id_aerial_duel,yellow_card,red_card,infraction_type,opponent_id_infraction,opponent_name_infraction,opponent_position_infraction,carry_progression,end_location_x_carry,end_location_y_carry,possession_id,possession_duration,possession_types,events_number,event_index,start_location_x,start_location_y,end_location_x_possession,end_location_y_possession,team_id_possession,team_name_possession,team_formation_possession,with_shot,with_shot_on_goal,with_goal,flank,xg_possession,back_pass,short_or_medium_pass,lateral_pass,forward_pass,progressive_pass,loose_ball_duel,carry,long_pass,pass_to_final_third,opportunity,head_pass,recovery,under_pressure,ball_out,loss,hand_pass,foul_suffered,ground_duel,linkup_play,offensive_duel,defensive_duel,foul,dribble,aerial_duel,counterpressing_recovery,pass,deep_completion,touch_in_box,interception,sliding_tackle,progressive_run,smart_pass,cross,deep_completed_cross,pass_to_penalty_area,second_assist,assist,key_pass,shot_assist,goal,conceded_goal,dribbled_past_attempt,shot_after_throw_in,shot_block,free_kick_cross,save,save_with_reflex,shot_after_corner,cross_blocked,through_pass,free_kick_shot,shot,head_shot,whistle,shot_after_free_kick,acceleration,penalty_foul,penalty_goal,penalty_conceded_goal,third_assist,penalty_save
366,366,1723218687,5476240,1H,22,1,00:22:01.220,1322.220738,acceleration,"progressive_run, carry",41,62,3157,Milan,4-3-3,3166,Bologna,4-1-3-2,344132,T. Hernández,LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,28.890,68.0,50.0,1.723219e+09,24.221151,"['attack', 'transition_medium']",12.0,5.0,43.0,96.0,86.0,50.0,3157.0,Milan,4-3-3,False,False,False,center,0.0000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
442,442,1723218785,5476240,1H,27,7,00:27:07.810,1628.810027,acceleration,"progressive_run, carry",47,32,3157,Milan,4-3-3,3166,Bologna,4-1-3-2,520698,T. Reijnders,LCMF3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,16.610,62.0,59.0,1.723219e+09,39.167364,['attack'],17.0,13.0,29.0,24.0,91.0,37.0,3157.0,Milan,4-3-3,True,True,False,right,0.2037,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
625,625,1723218988,5476240,1H,35,53,00:35:53.759,2154.759218,acceleration,"progressive_run, carry",36,20,3157,Milan,4-3-3,3166,Bologna,4-1-3-2,344132,T. Hernández,LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,23.620,57.0,33.0,1.723219e+09,15.622183,['attack'],10.0,2.0,29.0,63.0,93.0,0.0,3157.0,Milan,4-3-3,False,False,False,center,0.0000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
684,684,1723219075,5476240,1H,38,48,00:38:48.843,2329.

In [82]:
unique_types

array(['pass', 'duel', 'touch', 'shot', 'clearance', 'game_interruption',
       'corner', 'goalkeeper_exit', 'infraction', 'free_kick',
       'interception', 'throw_in', 'offside', 'shot_against', 'goal_kick',
       'acceleration', 'penalty', 'fairplay', 'own_goal'], dtype=object)

In [59]:
unique_secondary_types = {}
secondary_type_counts = {}

# Get unique primary_type values
unique_primary_types = df['primary_type'].unique()

for typ in unique_primary_types:
    data_frame = df[df['primary_type'] == typ]
    secondary_types_set = set()
    secondary_types_counter = {}

    for index, row in data_frame.iterrows():
        if pd.notna(row['secondary_types']):
            secondary_types = row['secondary_types'].split(', ')
            secondary_types_set.update(secondary_types)
            for sec_type in secondary_types:
                if sec_type in secondary_types_counter:
                    secondary_types_counter[sec_type] += 1
                else:
                    secondary_types_counter[sec_type] = 1

    unique_secondary_types[typ] = secondary_types_set
    secondary_type_counts[typ] = secondary_types_counter

print(unique_secondary_types, secondary_type_counts)

{'pass': {'third_assist', 'pass_to_penalty_area', 'assist', 'carry', 'counterpressing_recovery', 'deep_completion', 'deep_completed_cross', 'head_pass', 'key_pass', 'through_pass', 'smart_pass', 'progressive_run', 'hand_pass', 'pass_to_final_third', 'shot_assist', 'cross', 'under_pressure', 'long_pass', 'second_assist', 'linkup_play', 'loss', 'cross_blocked', 'progressive_pass', 'short_or_medium_pass', 'recovery', 'touch_in_box', 'forward_pass', 'lateral_pass', 'back_pass'}, 'duel': {'third_assist', 'dribbled_past_attempt', 'assist', 'carry', 'aerial_duel', 'counterpressing_recovery', 'sliding_tackle', 'opportunity', 'progressive_run', 'dribble', 'ground_duel', 'loose_ball_duel', 'shot_block', 'second_assist', 'linkup_play', 'foul_suffered', 'loss', 'offensive_duel', 'interception', 'recovery', 'defensive_duel'}, 'touch': {'third_assist', 'opportunity', 'under_pressure', 'assist', 'carry', 'progressive_run', 'second_assist', 'loss', 'touch_in_box'}, 'shot': {'shot_block', 'opportunity'

In [28]:
# Dictionary to store unique secondary types for each primary_type
unique_secondary_types = {}

# Iterate through each DataFrame in dfs
for typ, data_frame in dfs.items():
    # Initialize a set to collect unique secondary types
    secondary_types_set = set()
    
    # Iterate through each row in the current DataFrame
    for index, row in data_frame.iterrows():
        # Check if secondary_types is NaN
        if pd.notna(row['secondary_types']):
            # Split secondary_types string by comma and add to the set
            secondary_types = row['secondary_types'].split(', ')
            secondary_types_set.update(secondary_types)
    # Store unique secondary types in the dictionary
    unique_secondary_types[typ] = secondary_types_set

# Print unique secondary types for each primary_type
for typ, secondary_types in unique_secondary_types.items():
    print(f"Unique secondary types for primary_type '{typ}': {secondary_types}")

Unique secondary types for primary_type 'pass': {'third_assist', 'pass_to_penalty_area', 'assist', 'carry', 'counterpressing_recovery', 'deep_completion', 'deep_completed_cross', 'head_pass', 'key_pass', 'through_pass', 'smart_pass', 'progressive_run', 'hand_pass', 'pass_to_final_third', 'shot_assist', 'cross', 'under_pressure', 'long_pass', 'second_assist', 'linkup_play', 'loss', 'cross_blocked', 'progressive_pass', 'short_or_medium_pass', 'recovery', 'touch_in_box', 'forward_pass', 'lateral_pass', 'back_pass'}
Unique secondary types for primary_type 'duel': {'third_assist', 'dribbled_past_attempt', 'assist', 'carry', 'aerial_duel', 'counterpressing_recovery', 'sliding_tackle', 'opportunity', 'progressive_run', 'dribble', 'ground_duel', 'loose_ball_duel', 'shot_block', 'second_assist', 'linkup_play', 'foul_suffered', 'loss', 'offensive_duel', 'interception', 'recovery', 'defensive_duel'}
Unique secondary types for primary_type 'touch': {'third_assist', 'opportunity', 'under_pressure',

In [58]:
unique_secondary_types['duel']

{'aerial_duel',
 'assist',
 'carry',
 'counterpressing_recovery',
 'defensive_duel',
 'dribble',
 'dribbled_past_attempt',
 'foul_suffered',
 'ground_duel',
 'interception',
 'linkup_play',
 'loose_ball_duel',
 'loss',
 'offensive_duel',
 'opportunity',
 'progressive_run',
 'recovery',
 'second_assist',
 'shot_block',
 'sliding_tackle',
 'third_assist'}

In [55]:
# Function to calculate percentages
def calculate_secondary_type_percentages(df, unique_secondary_types):
    # Initialize dictionaries to store results
    secondary_type_percentage = {}
    co_occurrence_percentage = {}

    # Iterate over unique secondary types for the current primary type
    for sec_type in unique_secondary_types:
        # Calculate percentage of sec_type within the DataFrame
        count = df['secondary_types'].apply(lambda x: sec_type in x.split(',') if isinstance(x, str) else False).sum()
        percentage = (count / len(df)) * 100
        secondary_type_percentage[sec_type] = percentage

    # Calculate co-occurrence percentage between secondary types
    secondary_types_list = list(unique_secondary_types)
    for sec_type1, sec_type2 in combinations(secondary_types_list, 2):
        count_pair = (df[sec_type1] & df[sec_type2]).sum()
        sec_type1_count = df[sec_type1].sum()
        co_occurrence_percentage[f"{sec_type1}-{sec_type2}"] = (count_pair / sec_type1_count) * 100

    return secondary_type_percentage, co_occurrence_percentage


In [74]:
# Function to calculate percentages
def calculate_secondary_type_percentages(df, unique_secondary_types):
    # Initialize dictionaries to store results
    secondary_type_percentage = {}
    co_occurrence_percentage = {}

    # Iterate over unique secondary types for the current primary type
    for sec_type in unique_secondary_types:
        # Calculate percentage of sec_type within the DataFrame
        count = df['secondary_types'].apply(lambda x: sec_type in x.split(',') if isinstance(x, str) else False).sum()
        percentage = (count / len(df)) * 100
        secondary_type_percentage[sec_type] = percentage

    # Calculate co-occurrence percentage between secondary types
    secondary_types_list = list(unique_secondary_types)
    for sec_type1, sec_type2 in product(secondary_types_list, repeat=2):
        if sec_type1 != sec_type2:
            count_pair = (df[sec_type1] & df[sec_type2]).sum()
            sec_type1_count = df[sec_type1].sum()
            if sec_type1_count > 0:
                co_occurrence_percentage[f"{sec_type1}-{sec_type2}"] = (count_pair / sec_type1_count) * 100
            else:
                co_occurrence_percentage[f"{sec_type1}-{sec_type2}"] = 0.0

    return secondary_type_percentage, co_occurrence_percentage


In [56]:
print(unique_secondary_types['touch'])
secondary_type_percentage, co_occurrence_percentage = calculate_secondary_type_percentages(dfs['touch'], unique_secondary_types['touch'])
print(secondary_type_percentage)
print(co_occurrence_percentage)

{'third_assist', 'opportunity', 'under_pressure', 'assist', 'carry', 'progressive_run', 'second_assist', 'loss', 'touch_in_box'}
{'third_assist': 0.010378288620206528, 'opportunity': 0.3580509573971252, 'under_pressure': 0.3295106636915573, 'assist': 0.002594572155051632, 'carry': 52.51154584608998, 'progressive_run': 15.274246276788958, 'second_assist': 0.012972860775258159, 'loss': 9.970940791863422, 'touch_in_box': 1.4477712625188106}
{'third_assist-opportunity': 0.0, 'third_assist-under_pressure': 25.0, 'third_assist-assist': 0.0, 'third_assist-carry': 50.0, 'third_assist-progressive_run': 50.0, 'third_assist-second_assist': 0.0, 'third_assist-loss': 0.0, 'third_assist-touch_in_box': 0.0, 'opportunity-under_pressure': 0.0, 'opportunity-assist': 0.0, 'opportunity-carry': 1.680672268907563, 'opportunity-progressive_run': 0.0, 'opportunity-second_assist': 0.0, 'opportunity-loss': 42.016806722689076, 'opportunity-touch_in_box': 98.73949579831933, 'under_pressure-assist': 0.0, 'under_pr

In [83]:
secondary_type_percentage_pass, co_occurrence_percentage_pass = calculate_secondary_type_percentages(dfs['pass'], unique_secondary_types['pass'])
secondary_type_percentage_duel, co_occurrence_percentage_duel = calculate_secondary_type_percentages(dfs['duel'], unique_secondary_types['duel'])
secondary_type_percentage_touch, co_occurrence_percentage_touch = calculate_secondary_type_percentages(dfs['touch'], unique_secondary_types['touch'])
secondary_type_percentage_clearance, co_occurrence_percentage_clearance = calculate_secondary_type_percentages(dfs['clearance'], unique_secondary_types['clearance'])
secondary_type_percentage_game_interruption, co_occurrence_percentage_game_interruption = calculate_secondary_type_percentages(dfs['game_interruption'], unique_secondary_types['game_interruption'])
secondary_type_percentage_corner, co_occurrence_percentage_corner = calculate_secondary_type_percentages(dfs['corner'], unique_secondary_types['corner'])
secondary_type_percentage_goalkeeper_exit, co_occurrence_percentage_goalkeeper_exit = calculate_secondary_type_percentages(dfs['goalkeeper_exit'], unique_secondary_types['goalkeeper_exit'])
secondary_type_percentage_infraction, co_occurrence_percentage_infraction = calculate_secondary_type_percentages(dfs['infraction'], unique_secondary_types['infraction'])
secondary_type_percentage_free_kick, co_occurrence_percentage_free_kick = calculate_secondary_type_percentages(dfs['free_kick'], unique_secondary_types['free_kick'])
secondary_type_percentage_interception, co_occurrence_percentage_interception = calculate_secondary_type_percentages(dfs['interception'], unique_secondary_types['interception'])
secondary_type_percentage_throw_in, co_occurrence_percentage_throw_in = calculate_secondary_type_percentages(dfs['throw_in'], unique_secondary_types['throw_in'])
secondary_type_percentage_offside, co_occurrence_percentage_offside = calculate_secondary_type_percentages(dfs['offside'], unique_secondary_types['offside'])
secondary_type_percentage_shot_against, co_occurrence_percentage_shot_against = calculate_secondary_type_percentages(dfs['shot_against'], unique_secondary_types['shot_against'])
secondary_type_percentage_goal_kick, co_occurrence_percentage_goal_kick = calculate_secondary_type_percentages(dfs['goal_kick'], unique_secondary_types['goal_kick'])
secondary_type_percentage_acceleration, co_occurrence_percentage_acceleration = calculate_secondary_type_percentages(dfs['acceleration'], unique_secondary_types['acceleration'])
secondary_type_percentage_penalty, co_occurrence_percentage_penalty = calculate_secondary_type_percentages(dfs['penalty'], unique_secondary_types['penalty'])
secondary_type_percentage_fairplay, co_occurrence_percentage_fairplay = calculate_secondary_type_percentages(dfs['fairplay'], unique_secondary_types['fairplay'])
secondary_type_percentage_own_goal, co_occurrence_percentage_own_goal = calculate_secondary_type_percentages(dfs['own_goal'], unique_secondary_types['own_goal'])

In [76]:
print(secondary_type_percentage)
print(co_occurrence_percentage)

{'third_assist': 0.006903188867186707, 'pass_to_penalty_area': 0.02761275546874683, 'assist': 0.14659124594437653, 'carry': 0.022333846335015818, 'counterpressing_recovery': 0.0, 'deep_completion': 1.5065194527801578, 'deep_completed_cross': 0.0, 'head_pass': 1.498804124046243, 'key_pass': 0.03289166460247784, 'through_pass': 0.0, 'smart_pass': 0.0, 'progressive_run': 0.001218209800091772, 'hand_pass': 0.44017980776649357, 'pass_to_final_third': 0.3699297092945347, 'shot_assist': 0.0, 'cross': 3.406114601056594, 'under_pressure': 0.0, 'long_pass': 0.06903188867186708, 'second_assist': 0.005684979067094935, 'linkup_play': 0.523018074172734, 'loss': 1.1975002334902116, 'cross_blocked': 0.0, 'progressive_pass': 0.12344525974263287, 'short_or_medium_pass': 10.140784445897271, 'recovery': 0.7804664119254617, 'touch_in_box': 0.0, 'forward_pass': 30.98232377580067, 'lateral_pass': 33.962064946825144, 'back_pass': 14.756581378444997}
{'third_assist-pass_to_penalty_area': 6.2015503875969, 'thir

In [86]:
print('forward_pass-back_pass co-occurence percentage:', co_occurrence_percentage_pass['forward_pass-back_pass'])
print('forward_pass-lateral_pass co-occurence percentage:', co_occurrence_percentage_pass['forward_pass-lateral_pass'])

print('aerial_duel-loose_ball_duel co-occurence percentage:', co_occurrence_percentage_duel['aerial_duel-loose_ball_duel'])
print('aerial_duel-defensive_duel co-occurence percentage:', co_occurrence_percentage_duel['aerial_duel-defensive_duel'])

forward_pass-back_pass co-occurence percentage: 0.08625610452210314
forward_pass-lateral_pass co-occurence percentage: 0.0
aerial_duel-loose_ball_duel co-occurence percentage: 0.0
aerial_duel-defensive_duel co-occurence percentage: 0.0


In [ ]:
df_back_and_forward= df[(df['back_pass'] == 1) & (df['forward_pass'] == 1)]

In [57]:
for typ in unique_types:
    secondary_type_percentage, co_occurrence_percentage = calculate_secondary_type_percentages(dfs[typ], unique_secondary_types[typ])
    print(typ)
    print(secondary_type_percentage)
    print(co_occurrence_percentage)

pass
{'third_assist': 0.006903188867186707, 'pass_to_penalty_area': 0.02761275546874683, 'assist': 0.14659124594437653, 'carry': 0.022333846335015818, 'counterpressing_recovery': 0.0, 'deep_completion': 1.5065194527801578, 'deep_completed_cross': 0.0, 'head_pass': 1.498804124046243, 'key_pass': 0.03289166460247784, 'through_pass': 0.0, 'smart_pass': 0.0, 'progressive_run': 0.001218209800091772, 'hand_pass': 0.44017980776649357, 'pass_to_final_third': 0.3699297092945347, 'shot_assist': 0.0, 'cross': 3.406114601056594, 'under_pressure': 0.0, 'long_pass': 0.06903188867186708, 'second_assist': 0.005684979067094935, 'linkup_play': 0.523018074172734, 'loss': 1.1975002334902116, 'cross_blocked': 0.0, 'progressive_pass': 0.12344525974263287, 'short_or_medium_pass': 10.140784445897271, 'recovery': 0.7804664119254617, 'touch_in_box': 0.0, 'forward_pass': 30.98232377580067, 'lateral_pass': 33.962064946825144, 'back_pass': 14.756581378444997}
{'third_assist-pass_to_penalty_area': 6.2015503875969, 

For every classification combination a check if there are no doubles, doubles can be removed later: 

In [87]:
#We start with all combinations for duels:
print('aerial_duel-loose_ball_duel co-occurence percentage:', co_occurrence_percentage_duel['aerial_duel-loose_ball_duel'])
print('aerial_duel-defensive_duel co-occurence percentage:', co_occurrence_percentage_duel['aerial_duel-defensive_duel'])
print('loose_ball_duel-aerial_duel co-occurence percentage:', co_occurrence_percentage_duel['loose_ball_duel-aerial_duel'])
print('loose_ball_duel-defensive_duel co-occurence percentage:', co_occurrence_percentage_duel['loose_ball_duel-defensive_duel'])
print('defensive_duel-aerial_duel co-occurence percentage:', co_occurrence_percentage_duel['defensive_duel-aerial_duel'])
print('defensive_duel-loose_ball_duel co-occurence percentage:', co_occurrence_percentage_duel['defensive_duel-loose_ball_duel'])



aerial_duel-loose_ball_duel co-occurence percentage: 0.0
aerial_duel-defensive_duel co-occurence percentage: 0.0
loose_ball_duel-aerial_duel co-occurence percentage: 0.0
loose_ball_duel-defensive_duel co-occurence percentage: 0.0
defensive_duel-aerial_duel co-occurence percentage: 0.0
defensive_duel-loose_ball_duel co-occurence percentage: 0.0


In [109]:
pd.set_option.max_rows = 100
#All combinations for Passing
#Start with cross:
df_cross = df[(df['cross'] == 1)]
#print(len(df_cross))
print(df_cross['flank'].unique())
df_cross_non_flank = df_cross[df_cross['flank'].isna()]
#print(df_cross_non_flank)
print(df_cross.columns)
#print(df_cross['height'].unique())
df_cross_forward_backward = df_cross[(df_cross['forward_pass'] == 1) & (df_cross['back_pass'] == 1)]
print(len(df_cross_forward_backward))
df_cross_forward_lateral = df_cross[(df_cross['forward_pass'] == 1) & (df_cross['lateral_pass'] == 1)]
print(len(df_cross_forward_lateral))
df_cross_backward_lateral = df_cross[(df_cross['lateral_pass'] == 1) & (df_cross['back_pass'] == 1)]
print(len(df_cross_backward_lateral))
df_cross_lateral = df_cross[df_cross['lateral_pass'] == 1]
print(len(df_cross_lateral))
df_cross_lateral_smaller_12 = df_cross[(df_cross['lateral_pass']) == 1 & (df_cross['pass_length'] < 12)]
print('hoi')
print(len(df_cross_lateral_smaller_12))

df_cross_non_lateral = df_cross[(df_cross['lateral_pass'] == 0) & (df_cross['back_pass'] == 0) & (df_cross['forward_pass'] == 0) & (df_cross['pass_length'] > 12)]
print(df_cross_non_lateral)

Index(['Unnamed: 0', 'event_id', 'match_id', 'match_period', 'minute',
       'second', 'match_timestamp', 'video_timestamp', 'primary_type',
       'secondary_types',
       ...
       'shot', 'head_shot', 'whistle', 'shot_after_free_kick', 'acceleration',
       'penalty_foul', 'penalty_goal', 'penalty_conceded_goal', 'third_assist',
       'penalty_save'],
      dtype='object', length=142)
0
0
0
6361
hoi
1845
        Unnamed: 0    event_id  match_id match_period  minute  second  \
617            617  1723218835   5476240           1H      35      35   
2037          2037  1720900932   5476241           1H      14      39   
2144          2144  1720901061   5476241           1H      20      35   
2413          2413  1720901331   5476241           1H      40      47   
2573          2573  1720901626   5476241           2H      47      34   
...            ...         ...       ...          ...     ...     ...   
493732      493732  2139887464   5476537           2H      65      33   


In [39]:
secondary_type_percentage

{'third_assist': 0.006903188867186707,
 'pass_to_penalty_area': 0.02761275546874683,
 'assist': 0.14659124594437653,
 'carry': 0.022333846335015818,
 'counterpressing_recovery': 0.0,
 'deep_completion': 1.5065194527801578,
 'deep_completed_cross': 0.0,
 'head_pass': 1.498804124046243,
 'key_pass': 0.03289166460247784,
 'through_pass': 0.0,
 'smart_pass': 0.0,
 'progressive_run': 0.001218209800091772,
 'hand_pass': 0.44017980776649357,
 'pass_to_final_third': 0.3699297092945347,
 'shot_assist': 0.0,
 'cross': 3.406114601056594,
 'under_pressure': 0.0,
 'long_pass': 0.06903188867186708,
 'second_assist': 0.005684979067094935,
 'linkup_play': 0.523018074172734,
 'loss': 1.1975002334902116,
 'cross_blocked': 0.0,
 'progressive_pass': 0.12344525974263287,
 'short_or_medium_pass': 10.140784445897271,
 'recovery': 0.7804664119254617,
 'touch_in_box': 0.0,
 'forward_pass': 30.98232377580067,
 'lateral_pass': 33.962064946825144,
 'back_pass': 14.756581378444997}

Selecting events from duels: 
Split: aerial/defensive(offensive)/loose_ball: all three 1000 events

In [20]:
seed = 42
np.random.seed(seed)

In [22]:
df_duel = df[df['primary_type'] == 'duel']
df_duel_aerial = df_duel[df_duel['secondary_types'].str.contains('aerial_duel')]
df_duel_defensive = df_duel[df_duel['secondary_types'].str.contains('defensive_duel')]
df_duel_loose = df_duel[df_duel['secondary_types'].str.contains('loose_ball')]
number_of_rows_to_select = 1000
sample_aerial = df_duel_aerial.sample(number_of_rows_to_select, random_state=seed)
sample_defensive = df_duel_defensive.sample(number_of_rows_to_select, random_state=seed)
sample_loose = df_duel_loose.sample(number_of_rows_to_select, random_state=seed)

duel_sample = pd.concat([sample_aerial, sample_defensive, sample_loose])
duel_sample

c:\Users\IvovanMiert\.anaconda\Lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
c:\Users\IvovanMiert\.anaconda\Lib\site-packages\IPython\core\formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,Unnamed: 0,event_id,match_id,match_period,minute,second,match_timestamp,video_timestamp,primary_type,secondary_types,location_x,location_y,team_id,team_name,team_formation,opponent_team_id,opponent_team_name,opponent_team_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_length,recipient_id,recipient_name,recipient_position,end_location_x,end_location_y,shot_body_part,is_goal,on_target,goal_zone,xg,post_shot_xg,goalkeeper_action_id,goalkeeper,opponent_id_ground_duel,opponent_name_ground_duel,opponent_position_ground_duel,duel_type_ground_duel,kept_possession_ground_duel,progressed_with_ball_ground_duel,stopped_progress_ground_duel,recovered_possession_ground_duel,take_on_ground_duel,side_ground_duel,related_duel_id_ground_duel,opponent_id_aerial_duel,opponent_name_aerial_duel,opponent_position_aerial_duel,opponent_height_aerial_duel,first_touch_aerial_duel,height_aerial_duel,related_duel_id_aerial_duel,yellow_card,red_card,infraction_type,opponent_id_infraction,opponent_name_infraction,opponent_position_infraction,carry_progression,end_location_x_carry,end_location_y_carry,possession_id,possession_duration,possession_types,events_number,event_index,start_location_x,start_location_y,end_location_x_possession,end_location_y_possession,team_id_possession,team_name_possession,team_formation_possession,with_shot,with_shot_on_goal,with_goal,flank,xg_possession,back_pass,short_or_medium_pass,lateral_pass,forward_pass,progressive_pass,loose_ball_duel,carry,long_pass,pass_to_final_third,opportunity,head_pass,recovery,under_pressure,ball_out,loss,hand_pass,foul_suffered,ground_duel,linkup_play,offensive_duel,defensive_duel,foul,dribble,aerial_duel,counterpressing_recovery,pass,deep_completion,touch_in_box,interception,sliding_tackle,progressive_run,smart_pass,cross,deep_completed_cross,pass_to_penalty_area,second_assist,assist,key_pass,shot_assist,goal,conceded_goal,dribbled_past_attempt,shot_after_throw_in,shot_block,free_kick_cross,save,save_with_reflex,shot_after_corner,cross_blocked,through_pass,free_kick_shot,shot,head_shot,whistle,shot_after_free_kick,acceleration,penalty_foul,penalty_goal,penalty_conceded_goal,third_assist,penalty_save
459852,459852,2124454500,5476514,2H,53,13,00:53:13.698,3203.698858,duel,"aerial_duel, loss",84,56,3159,Juventus,5-4-1,3172,Atalanta,3-4-3,447804,F. Chiesa,LW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50007.0,B. Djimsiti,RCB3,190.0,False,176.0,2.124456e+09,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.124454e+09,35.908148,['transition_low'],16.0,15.0,30.0,15.0,84.0,56.0,3159.0,Juventus,5-4-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373222,373222,2080815062,5476461,2H,50,9,00:50:09.732,3089.732525,duel,aerial_duel,33,58,3315,Sassuolo,4-2-3-1,3166,Bologna,4-1-4-1,564143,K. Thorstvedt,RDMF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49991.0,R. Freuler,DMF,180.0,False,189.0,2.080815e+09,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.080815e+09,24.318341,"['set_piece_attack', 'attack', 'free_kick']",19.0,11.0,32.0,87.0,85.0,42.0,3166.0,Bologna,4-1-4-1,True,False,False,left,0.0173,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28141,28141,1743613469,5476256,2H,46,59,00:46:59.538,2991.538307,duel,aerial_duel,35,77,3178,Empoli,4-3-3,3236,Monza,3-4-2-1,435395,A. Ismajli,RCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426411.0,Dany Mota,CF,180.0,False,185.0,1.743614e+09,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.743614e+09,24.793063,['throw_in'],14.0,3.0,37.0,0.0,85.0,13.0,3236.0,Monza,3-4-2-1,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
348782,348782,2135

Selecting events from pass:
Split:

In [23]:
df_pass = df[df['primary_type'] == 'pass']
df_pass_cross = df_pass[df_pass['secondary_types'].str.contains('cross')]

In [25]:
number_of_rows_to_select = 1000
df_pass_cross_flank_left = df_pass_cross[df_pass_cross['flank'] == 'left'] 
sample_cross_left = df_pass_cross_flank_left.sample(number_of_rows_to_select)
df_pass_cross_flank_right = df_pass_cross[df_pass_cross['flank'] == 'right']
sample_cross_right = df_pass_cross_flank_right.sample(number_of_rows_to_select)
df_pass_cross_flank_center = df_pass_cross[df_pass_cross['flank'] == 'center']
cross_sample = pd.concat([sample_cross_left, sample_cross_right, df_pass_cross_flank_center])

In [28]:
df_pass_only = df_pass[~df_pass["secondary_types"].str.contains('cross')]
df_pass_only_short_or_medium = df_pass_only[df_pass_only['secondary_types'].str.contains('short_or_medium_pass')].sample(number_of_rows_to_select)
df_pass_only_long_pass = df_pass_only[df_pass_only['secondary_types'].str.contains('long_pass')].sample(number_of_rows_to_select)
df_pass_only_direction_forward = df_pass_only[df_pass_only['secondary_types'].str.contains('forward_pass')].sample(number_of_rows_to_select)
df_pass_only_direction_backward = df_pass_only[df_pass_only['secondary_types'].str.contains('back_pass')].sample(number_of_rows_to_select)
df_pass_only_direction_lateral = df_pass_only[~df_pass_only['secondary_types'].str.contains('forward_pass', 'back_pass')].sample(number_of_rows_to_select)



,Unnamed: 0,event_id,match_id,match_period,minute,second,match_timestamp,video_timestamp,primary_type,secondary_types,location_x,location_y,team_id,team_name,team_formation,opponent_team_id,opponent_team_name,opponent_team_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_length,recipient_id,recipient_name,recipient_position,end_location_x,end_location_y,shot_body_part,is_goal,on_target,goal_zone,xg,post_shot_xg,goalkeeper_action_id,goalkeeper,opponent_id_ground_duel,opponent_name_ground_duel,opponent_position_ground_duel,duel_type_ground_duel,kept_possession_ground_duel,progressed_with_ball_ground_duel,stopped_progress_ground_duel,recovered_possession_ground_duel,take_on_ground_duel,side_ground_duel,related_duel_id_ground_duel,opponent_id_aerial_duel,opponent_name_aerial_duel,opponent_position_aerial_duel,opponent_height_aerial_duel,first_touch_aerial_duel,height_aerial_duel,related_duel_id_aerial_duel,yellow_card,red_card,infraction_type,opponent_id_infraction,opponent_name_infraction,opponent_position_infraction,carry_progression,end_location_x_carry,end_location_y_carry,possession_id,possession_duration,possession_types,events_number,event_index,start_location_x,start_location_y,end_location_x_possession,end_location_y_possession,team_id_possession,team_name_possession,team_formation_possession,with_shot,with_shot_on_goal,with_goal,flank,xg_possession,back_pass,short_or_medium_pass,lateral_pass,forward_pass,progressive_pass,loose_ball_duel,carry,long_pass,pass_to_final_third,opportunity,head_pass,recovery,under_pressure,ball_out,loss,hand_pass,foul_suffered,ground_duel,linkup_play,offensive_duel,defensive_duel,foul,dribble,aerial_duel,counterpressing_recovery,pass,deep_completion,touch_in_box,interception,sliding_tackle,progressive_run,smart_pass,cross,deep_completed_cross,pass_to_penalty_area,second_assist,assist,key_pass,shot_assist,goal,conceded_goal,dribbled_past_attempt,shot_after_throw_in,shot_block,free_kick_cross,save,save_with_reflex,shot_after_corner,cross_blocked,through_pass,free_kick_shot,shot,head_shot,whistle,shot_after_free_kick,acceleration,penalty_foul,penalty_goal,penalty_conceded_goal,third_assist,penalty_save
152347,152347,1826541046,5476330,2H,51,12,00:51:12.564,3327.564216,pass,"back_pass, short_or_medium_pass",67,95,3254,Frosinone,4-2-3-1,3173,Cagliari,4-2-3-1,532674,M. Çuni,CF,True,-172.0,16.0,354123.0,Pol Lirola,RB,52.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.826541e+09,34.648304,"['attack', 'transition_low']",17.0,8.0,13.0,51.0,39.0,64.0,3254.0,Frosinone,4-2-3-1,False,False,False,center,0.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
184207,184207,1832229641,5476349,1H,0,59,00:00:59.000,60.000756,pass,"lateral_pass, short_or_medium_pass",23,66,3236,Monza,4-2-3-1,3194,Hellas Verona,3-5-2,20704,L. Caldirola,RCB,True,-68.0,36.0,272673.0,G. Kyriakopoulos,LB,35.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.832230e+09,99.478990,[],29.0,6.0,14.0,32.0,100.0,87.0,3236.0,Monza,4-2-3-1,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
213026,213026,1852275734,5476365,2H,73,7,01:13:07.876,4456.876907,pass,"lateral_pass, short_or_medium_pass",67,7,3161,Internazionale,5-3-2,3159,Juventus,3-5-2,435501,Carlos Augusto,LB5,True,122.0,31.0,130.0,S. de Vrij,CB,51.0,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.852276e+09,46.713301,"['counterattack', 'attack', 'transition_low']",24.0,10.0,4.0,16.0,66.0,64.0,3161.0,Internazionale,5-3-2,False,False,False,right,0.0000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,